In [9]:
import numpy as np
from astropy import units as u
from astropy import constants as const
from astropy.coordinates import SkyCoord
from sympy import symbols, Eq, solve

### Constants for directly observed properties of Sirius A and B

In [10]:
parallax = 0.37921 * u.arcsec
angular_separation = 7.4957 * u.arcsec
orbital_period = 50.1 * u.year

apparent_magnitude_A = -1.46
apparent_magnitude_B = 8.44

spectral_type_A = "A1V"
spectral_type_B = "DA2"


### Workout distance, magnitudes, and combined mass of system

In [12]:
# Compute distance via parallax
distance = parallax.to(u.parsec, equivalencies=u.parallax())

# Compute the absolute magnitudes of Sirius A and B from parallax distance
absolute_magnitude_A = apparent_magnitude_A - 5 * np.log10(distance.value) + 5
absolute_magnitude_B = apparent_magnitude_B - 5 * np.log10(distance.value) + 5

# Compute the luminosities of Sirius A and B
absolute_magnitude_Sun = 4.83
luminosity_A = const.L_sun * 10 ** ((absolute_magnitude_Sun - absolute_magnitude_A) / 2.5)
luminosity_B = const.L_sun * 10 ** ((absolute_magnitude_Sun - absolute_magnitude_B) / 2.5)

# Compute the semi-major axis of the orbit in parsecs
semi_major_axis_pc = (angular_separation.to(u.radian).value * distance).to(u.parsec)

# Convert the semi-major axis to astronomical units
semi_major_axis_au = semi_major_axis_pc.to(u.AU)

# Compute the combined mass of Sirius A and B using Kepler's third law
mass_A_plus_B = ((4*np.pi**2 * semi_major_axis_au**3) / (const.G * orbital_period**2)).to(u.M_sun)

# Print the results
print(f"Distance: {distance:.2f}")
print(f"Absolute Magnitude A: {absolute_magnitude_A:.2f}")
print(f"Absolute Magnitude B: {absolute_magnitude_B:.2f}")
print(f"Luminosity A: {luminosity_A:.2e}")
print(f"Luminosity B: {luminosity_B:.2e}")
print(f"Semi-major axis: {semi_major_axis_au}")
print(f"Combined Mass (A + B): {mass_A_plus_B.to(u.M_sun):.2f}")


Distance: 2.64 pc
Absolute Magnitude A: 1.43
Absolute Magnitude B: 11.33
Luminosity A: 8.73e+27 W
Luminosity B: 9.58e+23 W
Semi-major axis: 19.766620078584424 AU
Combined Mass (A + B): 3.08 solMass


### Work out other aspects of system

Assume we know the mass of Sirius A and B, work out the distance to center of gravity in the system (radius A and radius B)

First equation:
$$ m_a \times r_a = m_b \times r_b $$

Second Equation is to make centripidal force of Sirus A with Gravitation of the system.

$$ \frac{m_a v_a^2}{r_a} = \frac{G m_a m_b}{(r_a + r_b)^2}$$

Can solve for $ v_a $ by:

$$ P = \frac{2 \pi r_a}{v_a} $$

$$ v_a = \frac{2 \pi r_a}{P} $$

Substitute in for $ v_a $:

$$ \frac{m_a 4 \pi^2 r_a^2}{r_a P^2} = \frac{G m_a m_b}{(r_a + r_b)^2}$$

Solve for $ m_b $:

$$ m_b = \frac {4 pi^2}{G P^2} r_a (r_a + r_b)^2 $$

Two equations and two unknowns

In [5]:
# Assume we have mass of both Sirius A and B, find radius A and B
P = (orbital_period.to(u.s)).value # Orbital Period in Seconds.
mass_A = (2.063 * u.M_sun).to(u.kg).value # Mass of Sirius A
mass_B = (1.018 * u.M_sun).to(u.kg).value # Mass of Sirius B
G = const.G.value

rA, rB = symbols('rA rB')

In [13]:
# Two equations and Two Unknowns (rA and rB)
eq1 = Eq(mass_A*rA, mass_B*rB)
eq2 = Eq(mass_B, (4*np.pi**2/(G*P**2)) * rA * (rA + rB)**2)

In [14]:
solve((eq1,eq2), (rA, rB))

[(977458678388.455, 1980842095791.14),
 (-488729339194.228 - 846504046633.966*I,
  -990421047895.569 - 1715459575840.73*I),
 (-488729339194.228 + 846504046633.966*I,
  -990421047895.569 + 1715459575840.73*I)]

In [15]:
# The non imaginary number solution is therefore.
radius_A = 977458678388.455 * u.m
radius_B = 1980842095791.14 * u.m

# Verify the distance between the objects and compare to semi_major_axis_au
(radius_A + radius_B).to(u.au) - semi_major_axis_au

<Quantity 0.00839918 AU>